In [2]:
import nilearn

In [4]:
from nilearn import datasets


In [14]:
nilearn.datasets.fetch_adhd(n_subjects=40, data_dir='/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data', url=None, resume=True, verbose=1)


Dataset created in /mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd



 ...done. (1 seconds, 0 min)
Extracting data from /mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/74a52d3552145e11365bb68453d1a70c/adhd40_metadata.tgz..... done.


Downloaded 33095680 of 44414948 bytes (74.5%,    1.0s remaining) ...done. (4 seconds, 0 min)
Extracting data from /mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/166bfb3ae13f7c60c012aa2194eef088/adhd40_0010042.tgz..... done.


Downloaded 32858112 of 45583539 bytes (72.1%,    1.2s remaining) ...done. (4 seconds, 0 min)
Extracting data from /mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/166bfb3ae13f7c60c012aa2194eef088/adhd40_0010064.tgz..... done.


Downloaded 39878656 of 45461055 bytes (87.7%,    0.6s remaining) ...done. (5 seconds, 0 min)
Extracting data from /mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/166bfb3ae13f7c60c012aa2194eef088/adhd40_0010128.tgz..... done.


...

Downloaded 65511424 of 70396354 bytes (93.1%,    0.4s remaining) ...done. (7 seconds, 0 min)
Extracting data from /mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/166bfb3ae13f7c60c012aa2194eef088/adhd40_8409791.tgz..... done.


Downloaded 39682048 of 45075978 bytes (88.0%,    0.5s remaining) ...done. (5 seconds, 0 min)
Extracting data from /mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/166bfb3ae13f7c60c012aa2194eef088/adhd40_8697774.tgz..... done.


Downloaded 51945472 of 63380505 bytes (82.0%,    1.1s remaining) ...done. (6 seconds, 0 min)
Extracting data from /mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/166bfb3ae13f7c60c012aa2194eef088/adhd40_9744150.tgz..... done.


Downloaded 42876928 of 46607053 bytes (92.0%,    0.3s remaining) ...done. (5 seconds, 0 min)
Extracting data from /mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/166bfb3ae13f7c60c012aa2194eef088/adhd40_9750701.tgz..... done.


{'func': ['/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz',
  '/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0010064/0010064_rest_tshift_RPI_voreg_mni.nii.gz',
  '/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0010128/0010128_rest_tshift_RPI_voreg_mni.nii.gz',
  '/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0021019/0021019_rest_tshift_RPI_voreg_mni.nii.gz',
  '/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0023008/0023008_rest_tshift_RPI_voreg_mni.nii.gz',
  '/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0023012/0023012_rest_tshift_RPI_voreg_mni.nii.gz',
  '/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0027011/0027011_rest_tshift_RPI_voreg_mni.nii.gz',
  '/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0027018/0027018_rest_tshift_RPI_vor

In [29]:
from glob import glob
data_files = glob('data/adhd/data/**/*.csv', recursive=True)

In [30]:
data_files

['data/adhd/data/0010042/0010042_regressors.csv',
 'data/adhd/data/0010064/0010064_regressors.csv',
 'data/adhd/data/0010128/0010128_regressors.csv',
 'data/adhd/data/0021019/0021019_regressors.csv',
 'data/adhd/data/0023008/0023008_regressors.csv',
 'data/adhd/data/0023012/0023012_regressors.csv',
 'data/adhd/data/0027011/0027011_regressors.csv',
 'data/adhd/data/0027018/0027018_regressors.csv',
 'data/adhd/data/0027034/0027034_regressors.csv',
 'data/adhd/data/0027037/0027037_regressors.csv',
 'data/adhd/data/1019436/1019436_regressors.csv',
 'data/adhd/data/1206380/1206380_regressors.csv',
 'data/adhd/data/1418396/1418396_regressors.csv',
 'data/adhd/data/1517058/1517058_regressors.csv',
 'data/adhd/data/1552181/1552181_regressors.csv',
 'data/adhd/data/1562298/1562298_regressors.csv',
 'data/adhd/data/1679142/1679142_regressors.csv',
 'data/adhd/data/2014113/2014113_regressors.csv',
 'data/adhd/data/2497695/2497695_regressors.csv',
 'data/adhd/data/2950754/2950754_regressors.csv',


In [32]:
import pandas as pd

In [45]:
data_loaded = pd.read_csv(data_files[0], sep ='\t')

In [46]:
data_loaded.head(n=10)

,csf,constant,linearTrend,wm,global,motion-pitch,motion-roll,motion-yaw,motion-x,motion-y,motion-z,gm,compcor1,compcor2,compcor3,compcor4,compcor5
0,12140.708282,1.0,0.0,9322.722489,9955.469315,-0.0637,0.1032,-0.1516,-0.0376,-0.0112,0.0840,10617.938409,-0.035058,-0.006713,-0.071532,0.009847,-0.027601
1,12123.146913,1.0,1.0,9314.257684,9947.987176,-0.0708,0.0953,-0.1562,-0.0198,0.0021,0.0722,10611.036827,-0.026949,-0.091152,-0.030126,0.020055,-0.099798
2,12085.963127,1.0,2.0,9319.610045,9945.132852,-0.0795,0.0971,-0.1453,-0.0439,-0.0241,0.0972,10591.877177,0.002552,0.069165,0.090166,-0.016608,-0.071980
3,12109.299348,1.0,3.0,9299.841075,9943.648622,-0.0607,0.0918,-0.1601,-0.0418,-0.0133,0.0877,10592.008336,0.079391,0.029959,-0.098036,0.062493,0.024105
4,12072.330305,1.0,4.0,9297.870869,9925.640852,-0.0706,0.0873,-0.1482,-0.0313,-0.0118,0.0712,10570.445905,0.075471,-0.030123,0.084739,0.088217,0.012996
5,12044.996807,1.0,5.0,9309.868979,9913.370020,-0.0598,0.0952,-0.1448,-0.0454,-0.0132,0.0750,10548.447088,0.131266,0.043751,0.103500,0.065657,-0.010882
6,12087.832493,1.0,6.0,9310.241007,9905.494708,-0.0676,0.0942,-0.1547,-0.0552,-0.0065,0.0898,10535.495426,0.123215,-0.020961,0.009604,0.203578,-0.001587
7,12040.245975,1.0,7.0,9318.777316,9904.751785,-0.0758,0.0984,-0.1404,-0.0450,-0.0132,0.0968,10547.804147,0.033187,-0.004375,0.189657,0.085533,-0.035859
8,12090.141606,1.0,8.0,9316.941356,9932.068866,-0.0715,0.0939,-0.1288,-0.0517,-0.0186,0.1002,10578.658689,0.038133,0.086558,0.083204,-0.142444,0.027485
9,12128.086285,1.0,9.0,9325.494202,9947.690268,-0.0473,0.0853,-0.1425,-0.0486,-0.0117,0.0855,10595.251669,-0.029594,-0.003868,-0.063114,0.049362,-0.038318


In [56]:
data_file_pheno = glob('data/adhd/ADHD200_40subs_motion*.csv')
data_file_pheno

['data/adhd/ADHD200_40subs_motion_parameters_and_phenotypics.csv']

In [58]:
data_pheno = pd.read_csv(data_file_pheno[0])
data_pheno.head(n=10)

,Unnamed: 0,Subject,Rest.Scan,MeanFD,NumFD_greater_than_0.20,rootMeanSquareFD,FDquartile.top1.4thFD.,PercentFD_greater_than_0.20,MeanDVARS,MeanFD_Jenkinson,...,sess_1_rest_6_eyes,sess_1_anat_1,sess_1_which_anat,sess_2_rest_1,sess_2_rest_1_eyes,sess_2_rest_2,sess_2_rest_2_eyes,sess_2_anat_1,defacing_ok,defacing_notes
0,1,2014113,rest_1,0.0576,2,0.2400,0.0944,1.6000,16.1677,1.3868,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN
1,2,3902469,rest_1,0.0580,0,0.2409,0.0931,0.0000,17.4188,1.2040,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,4275075,rest_1,0.0789,0,0.2808,0.1520,0.0000,17.7796,1.8105,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,7774305,rest_1,0.0679,0,0.2606,0.1054,0.0000,16.7169,1.5137,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,yes,defaced part of front skull/brain
4,5,1019436,rest_1,0.0904,0,0.3006,0.1927,0.0000,19.6124,1.6769,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN
5,6,3699991,rest_1,0.1113,15,0.3337,0.2524,9.8039,19.8704,2.6121,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,3154996,rest_1,0.0881,0,0.2968,0.1619,0.0000,18.6120,1.8118,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,3884955,rest_1,0.0988,8,0.3144,0.1881,6.4000,17.3609,1.9543,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,27034,rest_1,0.0484,1,0.2201,0.0860,0.3831,21.5895,0.9262,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,4134561,rest_1,0.0439,4,0.2095,0.0849,1.5267,25.0028,0.9207,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN


In [130]:
data_pheno.columns

Index(['Unnamed: 0', 'Subject', 'Rest.Scan', 'MeanFD',
       'NumFD_greater_than_0.20', 'rootMeanSquareFD', 'FDquartile.top1.4thFD.',
       'PercentFD_greater_than_0.20', 'MeanDVARS', 'MeanFD_Jenkinson', 'site',
       'sibling_id', 'data_set', 'age', 'sex', 'handedness', 'full_2_iq',
       'full_4_iq', 'viq', 'piq', 'iq_measure', 'tdc', 'adhd',
       'adhd_inattentive', 'adhd_combined', 'adhd_subthreshold',
       'diagnosis_using_cdis', 'notes', 'sess_1_anat_2', 'oppositional',
       'cog_inatt', 'hyperac', 'anxious_shy', 'perfectionism',
       'social_problems', 'psychosomatic', 'conn_adhd', 'restless_impulsive',
       'emot_lability', 'conn_gi_tot', 'dsm_iv_inatt', 'dsm_iv_h_i',
       'dsm_iv_tot', 'study', 'sess_1_rest_1', 'sess_1_rest_1_eyes',
       'sess_1_rest_2', 'sess_1_rest_2_eyes', 'sess_1_rest_3',
       'sess_1_rest_3_eyes', 'sess_1_rest_4', 'sess_1_rest_4_eyes',
       'sess_1_rest_5', 'sess_1_rest_5_eyes', 'sess_1_rest_6',
       'sess_1_rest_6_eyes', 'sess_1_a

In [120]:
from numpy import loadtxt

sub_idx = list(loadtxt("data/adhd/ADHD200_40subs_ID.txt"))
print(sub_idx)

[2014113.0, 3902469.0, 4275075.0, 7774305.0, 1019436.0, 3699991.0, 3154996.0, 3884955.0, 27034.0, 4134561.0, 27018.0, 6115230.0, 27037.0, 8409791.0, 27011.0, 3007585.0, 8697774.0, 9750701.0, 10064.0, 21019.0, 10042.0, 10128.0, 2497695.0, 4164316.0, 1552181.0, 4046678.0, 23012.0, 1679142.0, 1206380.0, 23008.0, 4016887.0, 1418396.0, 2950754.0, 3994098.0, 3520880.0, 1517058.0, 9744150.0, 1562298.0, 3205761.0, 3624598.0]


In [122]:
sub_idx = [int(i) for i in sub_idx]
print(sub_idx)

[2014113, 3902469, 4275075, 7774305, 1019436, 3699991, 3154996, 3884955, 27034, 4134561, 27018, 6115230, 27037, 8409791, 27011, 3007585, 8697774, 9750701, 10064, 21019, 10042, 10128, 2497695, 4164316, 1552181, 4046678, 23012, 1679142, 1206380, 23008, 4016887, 1418396, 2950754, 3994098, 3520880, 1517058, 9744150, 1562298, 3205761, 3624598]


In [123]:
sub_idx = [str(i) for i in sub_idx]
print(sub_idx)

['2014113', '3902469', '4275075', '7774305', '1019436', '3699991', '3154996', '3884955', '27034', '4134561', '27018', '6115230', '27037', '8409791', '27011', '3007585', '8697774', '9750701', '10064', '21019', '10042', '10128', '2497695', '4164316', '1552181', '4046678', '23012', '1679142', '1206380', '23008', '4016887', '1418396', '2950754', '3994098', '3520880', '1517058', '9744150', '1562298', '3205761', '3624598']


In [125]:
# Path to resting state files
rest_files = ['/data/adhd/data/%s/%s_rest_tshift_RPI_voreg_mni.nii.gz' % (sub, sub) for sub in sub_idx]

In [127]:
# Path to counfound files
confound_files = ['/data/adhd/data/%s/%s_regressors.csv' % (sub, sub) for sub in sub_idx]